# N-BEATS (Darts) ? 1/7/30 kunlik forecast

- Target: rate; past_covariates: qolgan 23 ustun.
- Outlier clip, 70/15/15 split, MAE/MAPE.
- 1/7/30 forecast: covariates oxirgi qiymati bilan uzaytiriladi (agar kelajak ma'lum bo'lmasa).


In [1]:
print('hi')

hi


In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [15]:
# Ma'lumotlarni yuklash
p = Path('datasets/usd_rates_ready.csv')
df = pd.read_csv(p)
df.columns = [c.lower() for c in df.columns]
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Target = rate, date ishlatilmaydi, qolgan 23 ta ustun feature sifatida
feature_cols = [c for c in df.columns if c not in ['date', 'rate']]

# Outlierlarni IQR bilan yumshatish (faqat targetda)
def clip_iqr(s, k=1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - k * iqr, q3 + k * iqr
    return s.clip(lower, upper)

df['rate'] = clip_iqr(df['rate'])

# Vaqt bo'yicha train/val/test (70/15/15)
n = len(df)
train_end = int(n * 0.70)
val_end = int(n * 0.85)
train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

# Bashorat ufqlari
horizons = [1, 7, 30]

try:
    from darts import TimeSeries
    from darts.models import NBEATSModel
    from darts.dataprocessing.transformers import Scaler
except ImportError:
    raise SystemExit("Darts o'rnatilmagan: pip install darts[u]")

series = TimeSeries.from_dataframe(df, time_col='date', value_cols='rate')
covariates = TimeSeries.from_dataframe(df, time_col='date', value_cols=feature_cols)

scaler_y = Scaler()
scaler_x = Scaler()
series_s = scaler_y.fit_transform(series)
cov_s = scaler_x.fit_transform(covariates)

train, rest = series_s.split_before(0.70)
val, test = rest.split_before(0.50)

cov_train, cov_rest = cov_s.split_before(0.70)
cov_val, cov_test = cov_rest.split_before(0.50)

model = NBEATSModel(
    input_chunk_length=30,
    output_chunk_length=7,
    n_epochs=50,
    random_state=42,
)
model.fit(train, past_covariates=cov_train, val_series=val, val_past_covariates=cov_val)

val_pred = model.predict(n=len(val), series=train, past_covariates=cov_train.concatenate(cov_val))
test_pred = model.predict(n=len(test), series=train.concatenate(val), past_covariates=cov_train.concatenate(cov_val).concatenate(cov_test))

mae_val = mean_absolute_error(val.values(), val_pred.values())
mape_val = mean_absolute_percentage_error(val.values(), val_pred.values())
mae_test = mean_absolute_error(test.values(), test_pred.values())
mape_test = mean_absolute_percentage_error(test.values(), test_pred.values())
print(f"Val MAE: {mae_val:.2f}, MAPE: {mape_val:.4f}")
print(f"Test MAE: {mae_test:.2f}, MAPE: {mape_test:.4f}")


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 11.2 M | train
-------------------------------------------------------------
11.2 M    Trainable params
5.2 K     Non-trainable params
11.2 M    Total params
44.886    Total estimated model params size (MB)
396       Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Ai\AI_Rate_Prediction\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 49: 100%|██████████| 56/56 [00:13<00:00,  4.08it/s, train_loss=0.000157, val_loss=0.00101] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 56/56 [00:13<00:00,  4.08it/s, train_loss=0.000157, val_loss=0.00101]


`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.24it/s]

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.19it/s]
Val MAE: 0.03, MAPE: 0.0306
Test MAE: 0.05, MAPE: 0.0549


In [16]:
# 1/7/30 forecast: covariates oxirgi qiymati bilan uzaytiriladi
horizon_forecasts = {}
for h in horizons:
    future_dates = pd.date_range(df['date'].max() + pd.Timedelta(days=1), periods=h, freq='D')
    last_cov = cov_s.values()[-1]
    future_cov_vals = np.repeat(last_cov[np.newaxis, :], h, axis=0)
    future_cov = TimeSeries.from_times_and_values(future_dates, future_cov_vals)
    full_cov = cov_s.concatenate(future_cov)
    fc = model.predict(n=h, series=series_s, past_covariates=full_cov)
    fc_inv = scaler_y.inverse_transform(fc)
    horizon_forecasts[f'h_{h}'] = fc_inv.to_dataframe()

horizon_forecasts['h_30'].head(10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Ai\AI_Rate_Prediction\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 38.25it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Ai\AI_Rate_Prediction\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 36.13it/s]

`predict()` was called with `n > output_chunk_length`: using auto-regression to forecast the values after `output_chunk_length` points. The model will access `(n - output_chunk_length)` future values of your `past_covariates` (relative to the first predicted time step). To hide this warning, set `show_warnings=False`.
GPU available: False, used: False


TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Ai\AI_Rate_Prediction\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  9.50it/s]


,rate
date,
2025-12-10,12161.731626
2025-12-11,12138.751650
2025-12-12,12197.476153
2025-12-13,12225.645292
2025-12-14,12186.559802
2025-12-15,12140.395156
2025-12-16,12155.232678
2025-12-17,12175.441873
2025-12-18,12129.965709
